In [1]:
import pandas as pd 
import numpy as np

In [17]:
data = pd.read_csv('train.csv')
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [18]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [19]:
data.shape

(20800, 5)

In [20]:
data = data.dropna()

In [21]:
data.isnull().sum().sum()

0

Independant Feature 

In [22]:
X= data.drop('label',axis=1)
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [23]:
y= data['label']
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [24]:
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [26]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [27]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [28]:
voc_size = 5000

In [29]:
sentences = X.copy()

In [30]:
sentences['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [31]:
sentences.reset_index(inplace=True)

In [32]:
sentences

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [33]:
import nltk
import re
from nltk.corpus import stopwords

In [34]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
### Dataset Preprocessing
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(0, len(sentences)):
    review = re.sub('[^a-zA-Z]', ' ',sentences['title'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [36]:
corpus

['house dem aide even see comey letter jason chaffetz tweeted',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fired',
 'civilian killed single u airstrike identified',
 'iranian woman jailed fictional unpublished story woman stoned death adultery',
 'jackie mason hollywood would love trump bombed north korea lack trans bathroom exclusive video breitbart',
 'beno hamon win french socialist party presidential nomination new york time',
 'back channel plan ukraine russia courtesy trump associate new york time',
 'obama organizing action partner soros linked indivisible disrupt trump agenda',
 'bbc comedy sketch real housewife isi cause outrage',
 'russian researcher discover secret nazi military base treasure hunter arctic photo',
 'u official see link trump russia',
 'yes paid government troll social medium blog forum website',
 'major league soccer argentine find home success new york time',
 'well fargo chief abruptly step new york time',
 'anonymous donor pay 

In [37]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[148, 1017, 2840, 2039, 4908, 219, 619, 4053, 4598, 276],
 [2617, 1029, 2472, 3220, 4218, 2995, 4021],
 [2407, 2856, 2418, 69],
 [2125, 2303, 2692, 3049, 703, 1065],
 [2112, 4218, 1055, 4888, 1389, 688, 4218, 274, 2443, 3230],
 [3223,
  4415,
  351,
  1175,
  3424,
  3781,
  1564,
  1492,
  4072,
  118,
  1888,
  2946,
  2851,
  273,
  4021],
 [3977, 1300, 1524, 4381, 619, 3719, 3817, 3093, 1613, 1794, 3588],
 [3849, 224, 725, 820, 1567, 3407, 3781, 667, 1613, 1794, 3588],
 [4999, 1026, 1874, 2745, 2903, 3699, 3190, 3932, 3781, 3907],
 [1532, 92, 3459, 2781, 2116, 3586, 2202, 3430],
 [3514, 4366, 1415, 4720, 937, 1097, 640, 1809, 1867, 3154, 4524],
 [3049, 2492, 4908, 3480, 3781, 1567],
 [4469, 2946, 4604, 1065, 1936, 4450, 4251, 2146, 1352],
 [4055, 23, 208, 4412, 2547, 2721, 4044, 1613, 1794, 3588],
 [2988, 1582, 2803, 4315, 2814, 1613, 1794, 3588],
 [4629, 4645, 3736, 3297, 2138, 3728, 544, 4506, 3338, 1149],
 [4270, 2479, 1029],
 [3725, 844, 1034, 74, 3781, 3123, 4364, 4021],
 [89

In [38]:
## pre padding
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[ 148 1017 2840 ...    0    0    0]
 [2617 1029 2472 ...    0    0    0]
 [2407 2856 2418 ...    0    0    0]
 ...
 [1938 1847 2403 ...    0    0    0]
 [4191 1567  316 ...    0    0    0]
 [1100 2976 4039 ...    0    0    0]]


In [39]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [40]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [41]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [42]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10


383/383 [==============================] - 11s 15ms/step - loss: 0.2677 - accuracy: 0.8766 - val_loss: 0.1959 - val_accuracy: 0.9170
Epoch 2/10
383/383 [==============================] - 5s 13ms/step - loss: 0.1212 - accuracy: 0.9549 - val_loss: 0.2398 - val_accuracy: 0.9145
Epoch 3/10
383/383 [==============================] - 5s 12ms/step - loss: 0.0746 - accuracy: 0.9748 - val_loss: 0.2393 - val_accuracy: 0.9110
Epoch 4/10
383/383 [==============================] - 5s 12ms/step - loss: 0.0496 - accuracy: 0.9840 - val_loss: 0.3047 - val_accuracy: 0.9158
Epoch 5/10
383/383 [==============================] - 5s 12ms/step - loss: 0.0327 - accuracy: 0.9889 - val_loss: 0.3564 - val_accuracy: 0.9105
Epoch 6/10
383/383 [==============================] - 5s 12ms/step - loss: 0.0230 - accuracy: 0.9934 - val_loss: 0.4514 - val_accuracy: 0.8953
Epoch 7/10
383/383 [==============================] - 5s 12ms/step - loss: 0.0151 - accuracy: 0.9946 - val_loss: 0.5144 - val_accuracy: 0.9

In [43]:
y_pred=model.predict(X_test)

189/189 [==============================] - 2s 5ms/step


In [44]:

y_pred=np.where(y_pred>=0.5,1,0)

In [45]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[3151,  268],
       [ 278, 2338]], dtype=int64)

In [46]:
print(accuracy_score(y_test,y_pred))

0.9095277547638774


In [47]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3419
           1       0.90      0.89      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

